# IST 718: Big Data Analytics

- Professor: Willard Williamson <wewillia@syr.edu>
- Faculty Assistant: Palaniappan Muthukkaruppan
## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers from your classmates.  Short code snippets are allowed from the internet.  Any code is allowed from the class text books or class provided code.__
- Please do not change the file names. The FAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and FAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`).

In [2]:
# load these packages
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
from pyspark.ml.feature import PCA 
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
import pandas as pd
import os

The following cell is used to determine if the environment is databricks or personal computer and load the csv file accordingly.

In [4]:
def get_training_dataframe():
    data_file_name = "colleges_data_science_programs.csv"
    
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    grading_env = os.getenv("GRADING_RUNTIME_ENV")
    
    # if the databricks env var exists
    if db_env != None:
        full_path_name = "/FileStore/tables/%s" % data_file_name
    elif grading_env != None:
        full_path_name = "C:/Users/Will/Desktop/SU/datasets/%s" % data_file_name
    else:
        full_path_name = data_file_name
    
    return spark.read.csv(full_path_name, inferSchema=True, header=True).fillna('').orderBy('id')

ds_programs_df = get_training_dataframe

# Unsupervised learning

It is recommended to follow the notebook `unsupervised_learning.ipynb`.

The following dataset contains information about dozens of "data science" programs across the US.

In [7]:
ds_programs_df = get_training_dataframe()
ds_programs_df.head()

Out[3]: Row(id=1, name='South Dakota State University', url='http://www.sdstate.edu/mathstat/grad/masters-in-data-science.cfm', program='Data Science', degree='Masters', country='US', state='SD', online=False, oncampus=True, department='Mathematics and Statistics', created_at='2015-01-10 04:13:13 UTC', updated_at='2015-01-10 04:13:13 UTC', university_count=1, program_size='', courses='', admit_reqs='', year_founded=None, notes='', cost='', visualization='', machine learning='', business='', databases='', programminglanguages='', capstone='', mapreduce='', part-time='', ethics='')

## Question 1: (10 pts)

This dataset contains many columns that we can use to understand how these data science programs differ from one another. In this question, you will create a dataframe `ds_programs_text_df` which simply adds a column `text` to the dataframe `ds_programs_df`. This column will have the concatenation of the following columns separated by a space: `program`, `degree` and `department` (find the appropriate function in the `fn` package)

In [9]:
# (10 pts) Create ds_programs_text_df here
from pyspark.sql.functions import concat, col, lit

ds_programs_text_df = ds_programs_df.withColumn('text',concat(col("program"), lit(" "), col("degree"), lit(" "), col("department")))

#raise NotImplementedError()

An example of the `ds_programs_text_df` should give you:

```python
ds_programs_text_df.orderBy('id').first().text
```

```console
'Data Science Masters Mathematics and Statistics'
```

In [11]:
# (10 pts)
np.testing.assert_equal(ds_programs_text_df.count(), 222)
np.testing.assert_equal(set(ds_programs_text_df.columns), {'admit_reqs',
 'business',
 'capstone',
 'cost',
 'country',
 'courses',
 'created_at',
 'databases',
 'degree',
 'department',
 'ethics',
 'id',
 'machine learning',
 'mapreduce',
 'name',
 'notes',
 'oncampus',
 'online',
 'part-time',
 'program',
 'program_size',
 'programminglanguages',
 'state',
 'text',
 'university_count',
 'updated_at',
 'url',
 'visualization', 
 'year_founded'})
np.testing.assert_array_equal(ds_programs_text_df.orderBy('id').rdd.map(lambda x: x.text).take(5),
                              ['Data Science Masters Mathematics and Statistics',
 'Analytics Masters Business and Information Systems',
 'Data Science Masters Computer Science',
 'Business Intelligence & Analytics Masters Business',
 'Advanced Computer Science(Data Analytics) Masters Computer Science'])

# Question 2: (10 pts) 

The following code creates a dataframe `ds_features_df` which adds a column `features` to `ds_programs_text_df` that contains the `tfidf` of the column `text`:

In [13]:
# read-only
pipe_features = \
    Pipeline(stages=[
        feature.Tokenizer(inputCol='text', outputCol='words'),
        feature.CountVectorizer(inputCol='words', outputCol='tf'),
        feature.IDF(inputCol='tf', outputCol='tfidf'),
        feature.StandardScaler(withStd=False, withMean=True, inputCol='tfidf', outputCol='features')]).\
    fit(ds_programs_text_df)

Create a pipeline model `pipe_pca` that computes the two first principal components of `features` as computed by `pipe_features` and outputs a column `scores`. Use that pipeline to create a dataframe `ds_features_df` with the columns `id`, `name`, `url`, and `scores`.

In [15]:
# create the pipe_pca PipelineModel below (10 pts)
pca = PCA(k=2, inputCol="features", outputCol="scores")
pipe_pca = Pipeline(stages=[pipe_features, pca]).fit(ds_programs_text_df)
ds_features_df = pipe_pca.transform(ds_programs_text_df).select('id', 'name', 'url','scores')
#raise NotImplementedError()

In [16]:
# Tests for (10 pts)
np.testing.assert_equal(pipe_pca.stages[0],  pipe_features)
np.testing.assert_equal(type(pipe_pca.stages[1]),  feature.PCAModel)
np.testing.assert_equal(set(ds_features_df.columns), {'id', 'name', 'scores', 'url'})
np.testing.assert_equal(ds_features_df.first().scores.shape, (2, ))

# Question 3: (10 pts)

Create a scatter plot with the x axis containing the first principal component (loading) and the y axis containing the second principal component (loading) of `ds_features_df`

In [18]:
# below perform the appropriate 
loadings = pipe_pca.stages[-1].pc.toArray()
plt.figure(figsize=(8,8))
plot01 = plt.scatter(loadings[:,0], loadings[:,1], edgecolor='', alpha=0.5)
plt.title("PCA Scatter Plot")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
display(plot01.figure)
#raise NotImplementedError()

# Question 4 (10 pts)

Create two Pandas dataframes `pc1_pd` and `pc2_pd` with the columns `word` and `abs_loading` that contain the top 5 words in absolute loading for the principal components 1 and 2, respetively. You can extract the vocabulary from the stage that contains the count vectorizer in `pipe_features`:

In [20]:
# your code here
pc1_pd = pd.DataFrame({'word' : pipe_features.stages[1].vocabulary, 
                      'abs_loading' : np.abs(loadings[:,0])}).sort_values('abs_loading', ascending=False).head()
pc2_pd = pd.DataFrame({'word' : pipe_features.stages[1].vocabulary, 
                      'abs_loading' : np.abs(loadings[:,1])}).sort_values('abs_loading', ascending=False).head()
#raise NotImplementedError()

In [21]:
pc1_pd

,word,abs_loading
19,computational,0.440858
16,sciences,0.437846
13,of,0.389599
21,school,0.269706
41,"physics,",0.239209


In [22]:
pc2_pd

,word,abs_loading
7,computer,0.495368
5,science,0.405065
19,computational,0.317074
31,college,0.302729
2,business,0.201894


In [23]:
# (10 pts)
assert type(pc1_pd) == pd.core.frame.DataFrame
assert type(pc2_pd) == pd.core.frame.DataFrame
np.testing.assert_array_equal(pc1_pd.shape, (5, 2))
np.testing.assert_array_equal(pc2_pd.shape, (5, 2))
np.testing.assert_equal(set(pc1_pd.columns), {'abs_loading', 'word'})
np.testing.assert_equal(set(pc2_pd.columns), {'abs_loading', 'word'})

# Question 5: (10 pts)

Create a new pipeline for PCA called `pipe_pca2` where you fit 50 principal components. Extract the the `PCAModel` from the stages of this pipeline, and assign to a variable `explainedVariance` the variance explained by components of such model. Finally, assign to a variable `best_k` the value $k$ such that ($k+1$)-th component is not able to explain more than 0.01 variance. You can use a for-loop to find such best k.

In [25]:
# your code here
pca1 = PCA(k=50, inputCol="features", outputCol="scores")
pipe_pca2 = Pipeline(stages=[pipe_features, pca1]).fit(ds_programs_text_df)
explainedVariance = pipe_pca2.stages[-1].explainedVariance
#raise NotImplementedError()

In [26]:
k = 0
for k in range(len(explainedVariance)):
  if explainedVariance[k] < 0.01:
    print(k)
    break
best_k = k
best_k

32
Out[15]: 32

In [27]:
# Tests for (10 pts)
np.testing.assert_equal(pipe_pca2.stages[0],  pipe_features)
np.testing.assert_equal(type(pipe_pca2.stages[1]),  feature.PCAModel)
np.testing.assert_equal(len(explainedVariance), 50)
np.testing.assert_array_less(5, best_k)

# Question 6: (10 pts)

Create a new pipeline for PCA called pipe_pca3 where you fit all possible principal components for this dataset. Extract the the PCAModel from the stages of this pipeline, and use the object property named explainedVariance to create 2 separate plots:  A scree plot and a plot of cumulative variance explained.  Use the plots to check your code in the above question, is you answer from the above question believable?

In [29]:
# your code here
pca2 = PCA(k=118, inputCol="features", outputCol="scores")
pipe_pca3 = Pipeline(stages=[pipe_features, pca2]).fit(ds_programs_text_df)
#raise NotImplementedError()

In [30]:
plt.figure()
explained_var = pipe_pca3.stages[-1].explainedVariance
plot02 = plt.scatter(np.arange(1, len(explained_var)+1), explained_var)
plt.title("DS Programs Explained Variance")
plt.xlabel('Number of components')
plt.ylabel('Eexplained variance')
display(plot02.figure)

In [31]:
cum_sum = np.cumsum(explained_var)
plt.figure()
cumsum_plot = plt.scatter(np.arange(1, len(explained_var)+1), cum_sum)
plt.title("DS Programs Cumulative Sum of Explained Variance")
plt.xlabel("Cumulative Components")
plt.ylabel("Cumulative Sum of Variance Explained")
display(cumsum_plot.figure)

# Question 7: (10 pts)

Create a pipeline named k_means_pipe below by adding a feature.Normalizer and a clustering.KMeans object onto the end of the pipe_features pipeline object.  Set the seed of the KMeans clustering object to 2 and the number of clusters K to 5.  Transform the new pipe on the ds_programs_text_df dataframe creating a new dataframe named k_means_df.

In [34]:
# your code here
norm = feature.Normalizer(inputCol="features", outputCol="norm_features", p=2.0)
kmeans = clustering.KMeans(k=5, seed=2, featuresCol='norm_features', predictionCol='kmeans_clust')
k_means_pipe = Pipeline(stages=[pipe_features, norm, kmeans]).fit(ds_programs_text_df)
k_means_df = k_means_pipe.transform(ds_programs_text_df)
#raise NotImplementedError()

The `kmeans_clust` col in the resulting datafame contains the cluster assignment.  It's hard to visualize the clusters because we trained on all of the features in ds_programs_text_df.

In [36]:
# Show the head of the dataframe from above
display(k_means_df.toPandas().head())

id,name,url,program,degree,country,state,online,oncampus,department,created_at,updated_at,university_count,program_size,courses,admit_reqs,year_founded,notes,cost,visualization,machine learning,business,databases,programminglanguages,capstone,mapreduce,part-time,ethics,text,words,tf,tfidf,features,norm_features,kmeans_clust
1,South Dakota State University,http://www.sdstate.edu/mathstat/grad/masters-in-data-science.cfm,Data Science,Masters,US,SD,false,true,Mathematics and Statistics,2015-01-10 04:13:13 UTC,2015-01-10 04:13:13 UTC,1,,,,NaN,,,,,,,,,,,,Data Science Masters Mathematics and Statistics,"List(data, science, masters, mathematics, and, statistics)","List(0, 118, List(0, 1, 4, 5, 8, 36), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 118, List(0, 1, 4, 5, 8, 36), List(0.2538801769623398, 0.6709733230656233, 1.1586765294107597, 1.2962979072868075, 2.462732792293678, 3.615412302232064))","List(1, 118, List(), List(0.057180220036563034, 0.3052626379712971, -0.4954897991588664, -0.3629550481683585, 0.7672317559611788, 0.8641986048578716, -0.24098572336347812, -0.23897148177448113, 2.2186781912555658, -0.23897148177448113, -0.21909331387449488, -0.20638446153250645, -0.2086993319516678, -0.20406486596464488, -0.19968103721300093, -0.19710492462101956, -0.1792392513332439, -0.17021487380635458, -0.16209679163956386, -0.1590514375466797, -0.13555299543521387, -0.13555299543521387, -0.14459221595152696, -0.13555299543521387, -0.13555299543521387, -0.13984624677775104, -0.10492843066874764, -0.10492843066874764, -0.10492843066874764, -0.10492843066874764, -0.10492843066874764, -0.09771384600627199, -0.0935476114163461, -0.0935476114163461, -0.08142820500522666, -0.08142820500522666, 3.533984097226837, -0.0684276370995679, -0.0684276370995679, -0.0684276370995679, -0.05433618122081389, -0.05433618122081389, -0.05433618122081389, -0.05433618122081389, -0.05433618122081389, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.04246869000810967, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.03881585119632441, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835, -0.021234345004054835))","List(1, 118, List(), List(0.01277008198056407, 0.06817443007396702, -0.11065793996151045, -0.08105889969303649, 0.17134618256892756, 0.19300182868249513, -0.05381944038566742, -0.0533695989859018, 0.4954983099523972, -0.0533695989859018, -0.048930199600171496, -0.0460919263969152, -0.04660890735655772, -0.045573890167876475, -0.04459484789571575, -0.04401952361452673, -0.0400295754755202, -0.038014157543090996, -0.0362011429249488

In [37]:
np.testing.assert_equal(len(pd.unique(k_means_df.toPandas()['kmeans_clust'])), 5)

/databricks/spark/python/pyspark/sql/dataframe.py:2153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)

# Question 8: (10 pts)

The goal of this question is to make it easier to visualize the clusters by reducing the dimensionality of ds_programs_text_df to 2 dimensions using PCA.  Replace the Normalizer in the k_means_pipe above with a PCA object that reduces dimensionality to 2 dimensions. Save the PCA output scores in a column named `kmeans_clust`.  Transform the pipeline on ds_programs_text_df creating a new dataframe named k_means_pca_df. <br>

Define a function named get_kmeans_df which takes a parameter num_clusters, builds the pipeline described above using the parameter num_clusters to determine the number of KMeans clusters, and returns a pandas dataframe by transforming the pipeline on ds_programs_text_df.

In [39]:
# your code here
kmeans1 = clustering.KMeans(k=5, seed=2, featuresCol='scores', predictionCol='kmeans_clust')
k_means_pipe = Pipeline(stages=[pipe_features, pca, kmeans1]).fit(ds_programs_text_df)
k_means_pca_df = k_means_pipe.transform(ds_programs_text_df)
#raise NotImplementedError()

In [40]:
def get_kmeans_df(num_clusters):
  kmeans = clustering.KMeans(k=num_clusters, seed=2, featuresCol='scores', predictionCol='kmeans_clust')
  k_means_pipe = Pipeline(stages=[pipe_features, pca, kmeans]).fit(ds_programs_text_df)
  df = k_means_pipe.transform(ds_programs_text_df).toPandas()
  return df
  

In [41]:
# test
k_means_df = get_kmeans_df(3)
np.testing.assert_equal(k_means_df.shape[0], 222)
np.testing.assert_equal(k_means_df.shape[1], 35)
np.testing.assert_equal(len(pd.unique(k_means_df['kmeans_clust'])), 3)

/databricks/spark/python/pyspark/sql/dataframe.py:2153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)

# Question 9: (10 pts)

In the cell below, define a function named plot_kmeans which takes the dataframe returned by the get_kmeans_df function defined above as a single argument.  The plot_kmeans function shall creates a scatter plot of the PCA scores from the input dataframe where each score point is colored by the cluster assignment.  The plot title shall be the number of clusters in the dataframe.  The X and Y axes shall have descriptive labels.  The plot shall include a legend mapping colors to cluster number.  Feel free to use whatever plotting technique you like but the seaborn scatterplot function is one easy way of creating this function.

In [44]:
# your code here
def plot_kmeans(df):
  score1=[]
  score2=[]
  for i in range(len(df.scores)):
    score1.append(df.scores[i][0])
    score2.append(df.scores[i][1])
  df['score1'] = score1  
  df['score2'] = score2
  a = len(df["kmeans_clust"].unique()) 
  colors = df["kmeans_clust"]
  plt.figure(figsize=(8,8))
  plot01 = sns.scatterplot(df['score1'], df['score2'], hue=colors, size=colors, palette="Set1")
  plt.title("PCA Scatter Plot for " + str(a) + " Clusters")
  plt.xlabel("Principal Component 1")
  plt.ylabel("Principal Component 2")
  return display(plot01.figure)
#raise NotImplementedError()

In [45]:
print(sns.__version__)

0.9.0

In [46]:
# test your plot function here
# The plot should have 3 clusters, a title indicating that K=3, a legend, and labeled X and Y axes

plot_kmeans(get_kmeans_df(3))

Create 5 cluster plots picking various K of your choice below using get_kmeans and plot_kmeans.

In [48]:
# your code here
plot_kmeans(get_kmeans_df(4))
#raise NotImplementedError()

In [49]:
plot_kmeans(get_kmeans_df(2))

In [50]:
plot_kmeans(get_kmeans_df(6))

In [51]:
plot_kmeans(get_kmeans_df(9))

# Question 10: (10 pts)

Think about the cumulative variance and the scree plot from the question above.  What is one problem related to variance which might make the above KMeans not be an optimal choice using only 2 principal components.

Your Text Answer Here:
The above KMeans cannot be an optimal choice because of the amount of variance being captured by the first 2 principal components. Though the first two components posses the highest amount of variance in the data, we are still preserving less than 10% of the total variance of the data. In essence, we should preserve 90-95% of the total variance in data by choosing the number of components to be 50 so that we can preserve the variables that contribute most to the “cluster separation”.